# Planet API Python Client


This tutorial is an introduction to [Planet](https://www.planet.com)'s Data API using the official [Python client](https://github.com/planetlabs/planet-client-python), the `planet` module.

## Requirements

This tutorial assumes familiarity with the [Python](https://python.org) programming language throughout. Python modules used in this tutorial are:
* [IPython](https://ipython.org/) and [Jupyter](https://jupyter.org/)
* [planet](https://github.com/planetlabs/planet-client-python)
* [geojsonio](https://pypi.python.org/pypi/geojsonio)

You should also have an account on the Planet Platform and retrieve your API key from your [account page](https://www.planet.com/account/).

## Useful links 
* [Documentation](https://planetlabs.github.io/planet-client-python/index.html)
* [Planet Data API reference](https://www.planet.com/docs/reference/data-api/)

This tutorial will cover the basic operations possible with the Python client, particularly those that interact with the Data API.

The basic workflow for interacting with the Data API is:
1. search item types based on filters
1. activate assets
1. download assets

## Set up

In order to interact with the Planet API using the client, we need to import `planet.api`.

In [2]:
from planet import api

We next need to create a `ClientV1` object registered with our API key. The API key will be automatically read from the `PL_API_KEY` environment variable if it exists. If not it can be provided when creating the `api` object.

In [4]:
client = api.ClientV1()
# client = api.ClientV1(api_key="abcdef0123456789") <-- not a real key

#print client.auth.value

`ClientV1` provides basic low-level access to Planet’s API. Only one `ClientV1` should be in existence for an application. The client is thread safe and takes care to avoid API throttling and also retries any throttled requests. Any exceptional HTTP responses are handled by translation to one of the `planet.api.exceptions` classes.

We will also create a small helper function to print out JSON with proper indentation.

In [5]:
import json

def p(data):
    print(json.dumps(data, indent=2))

Let's also read in a GeoJSON geometry into a variable so we can use it during testing.

In [6]:
with open("Cairo.json") as f:
    geom = json.loads(f.read())

## Searching

We can search for items that are interesting by using the `quick_search` member function. Searches, however, always require a proper request that includes a filter that selects the specific items to return as seach results.

### Filters

The Planet Python client also includes `planet.api.filters` to assist in the creation of search filters.

In [ ]:
from planet.api import filters

The possible filters include `and_filter`, `date_filter`, `range_filter` and so on, mirroring the options supported by the Planet API.


In [ ]:
from datetime import datetime
start_date = datetime(year=2017, month=1, day=1)

date_filter = filters.date_range('acquired', gte=start_date)
cloud_filter = filters.range_filter('cloud_cover', lte=0.1)

In [ ]:
and_filter = filters.and_filter(date_filter, cloud_filter)

In [ ]:
p(and_filter)

In addition to the filter, a properly-constructed request also contains the list of item types that we want to select.

In [ ]:
item_types = ["REOrthoTile", "PSOrthoTile"]
req = filters.build_search_request(and_filter, item_types)

In [ ]:
p(req)

In [ ]:
res = client.quick_search(req)

The results of `quick_search` can be handled in different ways, but most commonly the user will either iterating through the list of items (`items_iter`) or writing items to a GeoJSON file (`json_encode`).

In either case, the number of items must be specified.

In [ ]:
for item in res.items_iter(4):
    print item['id'], item['properties']['item_type']

If the number of items requested is more than 250, the client will automatically fetch more pages of results in order to get the exact number requested.

In [ ]:
with open('results.geojson','w') as f:
    res.json_encode(f,1000)

This GeoJSON file can be opened and viewed in nay compatible application.

## Assets and downloads

After a search returns results, the Python client can be used to check for assets and initiate downloads.

The list of assets for an item can be retrieved with `get_assets` or `get_assets_by_id`.

In [ ]:
print item['id']

In [ ]:
assets = client.get_assets(item).get()

In [ ]:
for asset in sorted(assets.keys()):
    print asset

In [ ]:
activation = client.activate(assets['analytic'])
activation.response.status_code

A response of 202 means that the request has been accepted and the activation will begin shortly. A 204 code indicates that the asset is already active and no further action is needed. A 401 code means the user does not have permissions to download this file.

In [ ]:
assets = client.get_assets(item).get()

A response of 202 means that the request has been accepted and the activation will begin shortly. A 204 code indicates that the asset is already active and no further action is needed. A 401 code means the user does not have permissions to download this file.

In [ ]:
callback = api.write_to_file()
body = client.download(assets['analytic_xml'], callback=callback)
body.await()

## Saved Searches

The Python API client can also help in managing saved searches on the Planet Platform.

In [ ]:
searches = client.get_searches()

In [ ]:
for search in searches.items_iter(100):
    print search['id'], search['name']

In [ ]:
item_types = ["PSScene3Band"]
cairo_filter = filters.geom_filter(geom)
req = filters.build_search_request(cairo_filter, item_types, name="Egypt/Cairo governorate")

In [ ]:
p(req)

In [ ]:
res = client.create_search(req)

In [ ]:
search = res.get()
print search["id"], search["name"]

In [ ]:
res = client.saved_search(search["id"])

In [ ]:
for item in res.items_iter(20):
    print item["id"], item["properties"]["view_angle"]

# Statistics

The Python API client can also help report statistical summaries of the amount of data in the Planet API.

In [ ]:
item_types = ["PSScene3Band"]
cairo_filter = filters.geom_filter(geom)
req = filters.build_search_request(cairo_filter, item_types, interval="year")

In [ ]:
stats = client.stats(req).get()

In [ ]:
p(stats)

In [ ]:
assets = client.get_assets(item)

In [ ]:
assets.last_modified()

In [ ]:
assets.get()